In [24]:
import pandas as pd
import re
import chardet
from nltk.stem import WordNetLemmatizer
from thefuzz import fuzz
from thefuzz import process

In [25]:
# Clean and tokenize text
def preprocess_text(text):
    # Lowercase, removing special characters and numbers
    text = str(text).lower()
    text = re.sub(
        r"\s+", " ", text
    )  # Convert one or more of any kind of space to single space
    text = re.sub(r"[^a-z0-9\s]", "", text)  # Remove special characters
    text = text.strip()

# Fuzzy match given string to entire dataframe
def fuzzy_match_dataframe(df, match_string):
    # Fuzzy match element with given string
    def fuzzy_match(element):
        return fuzz.WRatio(match_string, str(element))

    # Prepare get scores and format result: (response, score, index)
    results = []
    for row in df.itertuples(index=True, name=None):
        row_index = row[0]
        for col_index, response in enumerate(row[1:]):
            score = fuzzy_match(response)
            # Append a dictionary for each matching record
            results.append({'response': response, 'score': score}) # can add ```'row_index': row_index, 'col_index': col_index```

    # Convert list of dictionaries to DataFrame
    result_df = pd.DataFrame(results)

    return result_df

In [26]:
# ------------------ Load and process data ------------------
file_name_1 = 'SST Member Viewership Study - B3a open ends.csv'
file_name_2 = 'SST Member Viewership Study - B4a open ends.csv'
file_name_3 = 'SST Member Viewership Study - Top titles to match.csv'

print("Loading data...")
with open(file_name_1, 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']
df_tv = pd.read_csv(file_name_1, encoding=encoding)

with open(file_name_2, 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']
df_movies = pd.read_csv(file_name_2, encoding=encoding)

with open(file_name_3, 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']
df_titles = pd.read_csv(file_name_3, encoding=encoding)

print("Pre-processing data...")
df_tv_processed = df_tv.iloc[:, 1:].map(preprocess_text)
df_movies_processed = df_movies.iloc[:, 1:].map(preprocess_text)

Loading data...
Pre-processing data...


In [27]:
# ------------------ Fuzzy match ------------------
print("Fuzzy matching...")
string_to_match = "Halo"
fuzzy_match_Halo = fuzzy_match_dataframe(df_tv_processed, string_to_match)

Fuzzy matching...


In [28]:
fuzzy_match_Halo_sorted = fuzzy_match_Halo.sort_values(by='score', ascending=False)
print("Sorted matches:\n", fuzzy_match_Halo_sorted)

Sorted matches:
        response  score
65917      halo    100
176397     halo    100
200370     halo    100
124956     halo    100
227898     halo    100
...         ...    ...
59630      1923      0
25454    dexter      0
59629      1883      0
191610     1893      0
90934      1923      0

[232404 rows x 2 columns]


In [29]:
fuzz_threshold = 60
fuzzy_matched_halo_50plus = fuzzy_match_Halo[fuzzy_match_Halo['score'] >= fuzz_threshold]
print("Highscore matches:\n", fuzzy_matched_halo_50plus)

Highscore matches:
                                response  score
24                                 halo    100
162     el rey de tulsa halo yelowstone     90
306                                halo    100
312                                halo    100
349                      hallos iramban     77
...                                 ...    ...
232045                             halo    100
232068                         house md     60
232296                             halo    100
232326                             halo    100
232392                             halo    100

[3542 rows x 2 columns]


In [30]:
# Initialize an empty dictionary to hold each DataFrame
df_all_coded = {}

for title in df_titles['TV Shows']:
    # Get the result from the fuzzy_match_dataframe function
    fuzzy_match_results = fuzzy_match_dataframe(df_tv_processed, title)
    results_highscore = fuzzy_match_results[fuzzy_match_results['score'] >= fuzz_threshold]

    # Create a new DataFrame for each title
    df_coded = pd.DataFrame()
    df_coded[title] = results_highscore['response']
    df_coded[f"{title}_score"] = results_highscore['score']

    # Store the DataFrame in the dictionary
    df_all_coded[title] = df_coded


In [31]:
print("Highscore matches against '1923':\n", df_all_coded['1923'])

Highscore matches against '1923':
         1923  1923_score
109     1923         100
240     1923         100
463     1923         100
554     1923         100
691     1923         100
...      ...         ...
231612  1923         100
232268  1923         100
232280  1923         100
232291  1923         100
232399  1923         100

[1372 rows x 2 columns]
